In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import re
import warnings
from sklearn.model_selection import  train_test_split
from sklearn import metrics
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, accuracy_score




In [18]:
from google.colab import files

# Choose the file from your local machine using the file picker dialog
uploaded = files.upload()

Saving instagram_reach.csv to instagram_reach.csv


In [51]:
file_name = next(iter(uploaded))

# Read the CSV file into a Pandas DataFrame
data = pd.read_csv(file_name)

# Display the first few rows of the DataFrame
data.head()

,Unnamed: 0,S.No,USERNAME,Caption,Followers,Hashtags,Time since posted,Likes
0,0,1,mikequindazzi,Who are #DataScientist and what do they do? >>...,1600,#MachineLearning #AI #DataAnalytics #DataScien...,11 hours,139
1,1,2,drgorillapaints,We all know where it’s going. We just have to ...,880,#deck .#mac #macintosh#sayhello #apple #steve...,2 hours,23
2,2,3,aitrading_official,Alexander Barinov: 4 years as CFO in multinati...,255,#whoiswho #aitrading #ai #aitradingteam#instat...,2 hours,25
3,3,4,opensourcedworkplace,sfad,340,#iot #cre#workplace #CDO #bigdata #technology#...,3 hours,49
4,4,5,crea.vision,Ever missed a call while your phone was chargi...,304,#instamachinelearning #instabigdata#instamarke...,3 hours,30


In [22]:
data.drop(['S.No','Unnamed: 0','USERNAME','Caption','Hashtags'],axis=1,inplace=True)

In [23]:
data.isnull().sum()

Followers            0
Time since posted    0
Likes                0
dtype: int64

In [24]:
data['Time since posted'] = data['Time since posted'].map(lambda a: int(re.sub('hours', '', a)))

In [26]:
X = data.drop(['Likes', 'Time since posted'], axis=1)
y = data[['Likes', 'Time since posted']]

In [27]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.preprocessing import StandardScaler, MinMaxScaler,RobustScaler
from sklearn.multioutput import MultiOutputRegressor


In [28]:
from sklearn.ensemble import AdaBoostRegressor, GradientBoostingRegressor, RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np
import pandas as pd

class ModelEvaluator:
    def __init__(self, models):
        self.models = models

    def evaluate_clf(self, true, predicted):
        MAE = mean_absolute_error(true, predicted)
        MSE = mean_squared_error(true, predicted)
        RMSE = np.sqrt(MSE)
        r2score = r2_score(true, predicted)
        return MAE, MSE, RMSE, r2score

    def evaluate_models(self, X, y):
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

        scalar = StandardScaler()
        X_train = scalar.fit_transform(X_train)
        X_test = scalar.transform(X_test)

        models_list = []
        MAE_train_list = []
        MAE_test_list = []
        MSE_train_list = []
        MSE_test_list = []
        RMSE_train_list = []
        RMSE_test_list = []
        r2score_train_list = []
        r2score_test_list = []
        r2score_diff = []

        for model_name, model in self.models.items():
            wrapper = MultiOutputRegressor(model)
            wrapper.fit(X_train, y_train)

            y_train_pred = wrapper.predict(X_train)
            y_test_pred = wrapper.predict(X_test)

            model_train_MAE, model_train_MSE, model_train_RMSE, model_train_r2score = self.evaluate_clf(y_train, y_train_pred)

            MAE_train_list.append(model_train_MAE)
            MSE_train_list.append(model_train_MSE)
            RMSE_train_list.append(model_train_RMSE)
            r2score_train_list.append(model_train_r2score * 100)

            model_test_MAE, model_test_MSE, model_test_RMSE, model_test_r2score = self.evaluate_clf(y_test, y_test_pred)

            MAE_test_list.append(model_test_MAE)
            MSE_test_list.append(model_test_MSE)
            RMSE_test_list.append(model_test_RMSE)
            r2score_test_list.append(model_test_r2score * 100)

            r2score_diff.append(max(model_train_r2score, model_test_r2score) - min(model_train_r2score, model_test_r2score))

            models_list.append(model_name)

            print(model_name)
            print('Model performance for Training set')
            print("- MAE: {:.4f}".format(model_train_MAE))
            print('- MSE: {:.4f}'.format(model_train_MSE))
            print('- RMSE: {:.4f}'.format(model_train_RMSE))
            print('- r2score: {:.4f}'.format(model_train_r2score))

            print('----------------------------------')

            print('Model performance for Test set')
            print('- MAE: {:.4f}'.format(model_test_MAE))
            print('- MSE: {:.4f}'.format(model_test_MSE))
            print('- RMSE: {:.4f}'.format(model_test_RMSE))
            print('- r2score: {:.4f}'.format(model_test_r2score))

            print('=' * 35)
            print('\n')

        report = pd.DataFrame(list(
            zip(models_list, MAE_train_list, MAE_test_list, MSE_train_list, MSE_test_list, RMSE_train_list,
                RMSE_test_list, r2score_train_list, r2score_test_list, r2score_diff)),
            columns=['Model Name', 'MAE_train', 'MAE_test', 'MSE_train', 'MSE_test', 'RMSE_train', 'RMSE_test',
                     'r2score_train', 'r2score_test', 'r2score_diff']).sort_values(by='r2score_test', ascending=False)

        return report

# Example usage:
models = {
    "AdaBoost Regressor": AdaBoostRegressor(),
    "Gradient Boosting": GradientBoostingRegressor(),
    "XGBRegressor": XGBRegressor(),
    "Decision Tree Regressor": DecisionTreeRegressor(),
    "Random Forest Regressor": RandomForestRegressor()
}

# Assuming you have your X and y defined
# X, y = ...

evaluator = ModelEvaluator(models)
result_report = evaluator.evaluate_models(X, y)
print(result_report)



AdaBoost Regressor
Model performance for Training set
- MAE: 15.4925
- MSE: 773.7132
- RMSE: 27.8157
- r2score: 0.6993
----------------------------------
Model performance for Test set
- MAE: 13.8230
- MSE: 663.3973
- RMSE: 25.7565
- r2score: 0.1153


Gradient Boosting
Model performance for Training set
- MAE: 4.4906
- MSE: 56.4654
- RMSE: 7.5143
- r2score: 0.9697
----------------------------------
Model performance for Test set
- MAE: 5.7064
- MSE: 95.7683
- RMSE: 9.7861
- r2score: 0.5670


XGBRegressor
Model performance for Training set
- MAE: 0.5929
- MSE: 2.6355
- RMSE: 1.6234
- r2score: 0.9931
----------------------------------
Model performance for Test set
- MAE: 12.0721
- MSE: 772.2032
- RMSE: 27.7885
- r2score: -0.0427


Decision Tree Regressor
Model performance for Training set
- MAE: 0.3438
- MSE: 2.4469
- RMSE: 1.5642
- r2score: 0.9932
----------------------------------
Model performance for Test set
- MAE: 5.9000
- MSE: 97.5500
- RMSE: 9.8767
- r2score: 0.4991


Random For

In [45]:
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from scipy.stats import randint

def perform_hyperparameter_tuning_random(X, y):
    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Define the Random Forest Regressor
    rf = RandomForestRegressor()

    # Define the hyperparameter distribution
    param_dist = {
        'n_estimators': randint(50, 200),
        'max_depth': [None, 10, 20],
        'min_samples_split': randint(2, 10),
        'min_samples_leaf': randint(1, 4)
    }

    # Create the RandomizedSearchCV object
    random_search = RandomizedSearchCV(estimator=rf, param_distributions=param_dist,
                                       scoring='neg_mean_squared_error', cv=5, n_iter=100, random_state=42)

    # Fit the random search to the data
    random_search.fit(X_train, y_train)

    # Get the best hyperparameters
    best_params = random_search.best_params_
    print("Best Hyperparameters:", best_params)

    # Use the best model for predictions
    best_model = random_search.best_estimator_
    y_pred = best_model.predict(X_test)

    # Evaluate the model
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    print("Mean Squared Error on Test Set:", mse)
    print("R-squared (R2) Score on Test Set:", r2)

    return best_model, best_params, mse, r2

# Example usage:
# Assuming you have your X and y defined
# X, y = ...

best_model, best_params, mse, r2 = perform_hyperparameter_tuning_random(X, y)




Best Hyperparameters: {'max_depth': 20, 'min_samples_leaf': 3, 'min_samples_split': 9, 'n_estimators': 172}
Mean Squared Error on Test Set: 396.82875764106257
R-squared (R2) Score on Test Set: 0.3206274850128035


In [47]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Assuming you have your X and y defined
# X, y = ...

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

# Assuming you have your X and y defined
# X, y = ...

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a RandomForestRegressor with specified hyperparameters
rf = RandomForestRegressor(
    max_depth=20,
    min_samples_leaf=3,
    min_samples_split=9,
    n_estimators=172
)

# Fit the model to the training data
rf.fit(X_train, y_train)

# Make predictions on the test set
y_pred = rf.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean Squared Error on Test Set:", mse)
print("R-squared (R2) Score on Test Set:", r2)



Mean Squared Error on Test Set: 394.3282778629438
R-squared (R2) Score on Test Set: 0.336791296631316
